In [3]:
# !pip3 install langchain
!pip3 install langgraph

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/airhorns/python/pypi/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 639.6 kB/s eta 0:00:001m660.7 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 729.8 kB/s eta 0:00:001m1.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 485.6 kB/s eta 0:00:000:00:01m eta 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [16]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, Tool
from langchain.schema import HumanMessage
from langgraph.graph import StateGraph
from pydantic import BaseModel

In [9]:
# setting openai api key
os.environ["OPENAI_API_KEY"] = "sk-proj-**********************************************"

In [7]:
# defining roles and prompts for the agents
research_prompt = ChatPromptTemplate.from_template(
    """Act as research assitant. Find the relevant information on: {query}"""
) 

writer_prompt = ChatPromptTemplate.from_template(
    """Act as content writer. Summarize the research results: '{research_response}' into a concise answer for: {query}"""
)

In [10]:
# creating LLM instances for each agent separately
research_agent = ChatOpenAI(model_name="gpt-4")
writer_agent = ChatOpenAI(model_name="gpt-4")

In [63]:
# Define functions for research agent
def research_agent_utility(state: MyState)-> dict:
    print(f"============================ {state}")
    if state is None or "query" not in state:
        return {"research_agent_response": "Error: No query found"}
    query = state.query
    response = research_agent([HumanMessage(content=research_prompt.format(query=query))])
    return {"research_agent_response": response.content}

In [71]:
# Define function for writer agent
def writer_agent_utility(state: MyState) -> dict:
    query = state.query
    research_response = state.research_agent_response
    response = writer_agent([HumanMessage(content=writer_prompt.format(query=query, research_response=research_response))])
    return {"final_response": response.content}

In [72]:
# Define state schema using Pydantic
class MyState(BaseModel):
    query: str  # Input query
    research_agent_response: str = None  # Output from research agent
    final_response: str = None

In [73]:
workflow = StateGraph(state_schema=MyState)
workflow.add_node("research_agent", research_agent_utility) # adding research agent as graph node
workflow.add_node("writer_agent", writer_agent_utility) # adding writer agent as graph node
# model agents can be added here, so this part is dynamic


In [74]:
# Defining execution workflow
workflow.set_entry_point("research_agent")
workflow.add_edge("research_agent", "writer_agent")

In [75]:
# Compiling the workflow defined above
graph = workflow.compile()

In [76]:
def multi_agent_executor(query):
    """ 
        Runs the multi agent collaborative system     
    """
    input_data = {"query": query}
    output = graph.invoke(input_data)
    return output["final_response"]


In [78]:

# query = "What is the average tempratur for the past decade of Pakistan?"
query = "Generate a chart of average temprature in Alaska over the past decade"
answer = multi_agent_executor(query)
print(answer)

============================ query='Generate a chart of average temprature in Alaska over the past decade' research_agent_response=None final_response=None
The research data reveals that the average temperature in Alaska over the past decade has been gradually increasing. The chart generated illustrates a clear upward trend, indicating the warming climate. The coldest average temperatures were recorded at the beginning of the decade, with a notable increase observed from the mid to the end of the decade. This data is consistent with the global trend of rising temperatures, reinforcing concerns about climate change.
